In [1]:
import requests
import pandas as pd
import datetime

In [ ]:
PLAID_CLIENT_ID = "67ccc3b7a291e80023d19c1f"
PLAID_SECRET = "38edeef47946602f3790b709d3ea9c"
PLAID_ENV = "sandbox"

In [ ]:
import requests, json

sandbox_request = {
    "client_id": PLAID_CLIENT_ID,
    "secret": PLAID_SECRET,
    "institution_id": "ins_109508",
    "initial_products": ["transactions", "identity"]
}
response = requests.post(f"https://sandbox.plaid.com/sandbox/public_token/create", json=sandbox_request)
sandbox_data = response.json()
print("Public token:", sandbox_data.get("public_token"))

In [ ]:
exchange_request = {
    "client_id": PLAID_CLIENT_ID,
    "secret": PLAID_SECRET,
    "public_token": sandbox_data["public_token"]
}
response = requests.post(f"https://sandbox.plaid.com/item/public_token/exchange", json=exchange_request)
exchange_data = response.json()
access_token = exchange_data.get("access_token")
item_id = exchange_data.get("item_id")
print("Access token:", access_token)
print("Item ID:", item_id)

In [7]:
import requests
import pandas as pd
import datetime
import json
from pprint import pprint

client_id = "67ccc3b7a291e80023d19c1f"
secret = "38edeef47946602f3790b709d3ea9c"
access_token = "access-sandbox-8259a179-18bf-4da1-a4eb-6e3615437a5b"
environment = "sandbox"

base_url = f"https://{environment}.plaid.com"

auth_payload = {
    "client_id": client_id,
    "secret": secret,
    "access_token": access_token
}

def plaid_post(endpoint, payload):
    """Make a POST request to Plaid API and return JSON data. Handle errors gracefully."""
    try:
        response = requests.post(base_url + endpoint, json=payload)
    except Exception as e:
        print(f"Network error when calling {endpoint}: {e}")
        return None
    if response.status_code != 200:
        try:
            err_json = response.json()
            err_msg = err_json.get('error_message', response.text)
        except ValueError:
            err_msg = response.text
        print(f"API error {response.status_code} on {endpoint}: {err_msg}")
        return None
    return response.json()

accounts_resp = plaid_post("/accounts/get", auth_payload)
if not accounts_resp:
    raise SystemExit("Failed to retrieve accounts. Check credentials/access token.")
accounts_data = accounts_resp.get("accounts", [])
print(f" → Retrieved {len(accounts_data)} accounts.")

end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=730)
transactions_data = []
page_size = 500
offset = 0

while True:
    tx_payload = {
        **auth_payload,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "options": {"count": page_size, "offset": offset}
    }
    tx_resp = plaid_post("/transactions/get", tx_payload)
    if not tx_resp:
        print("Failed to fetch transactions; skipping.")
        break
    tx_batch = tx_resp.get("transactions", [])
    total_tx = tx_resp.get("total_transactions", len(tx_batch))

    transactions_data.extend(tx_batch)
    offset += len(tx_batch)

    if len(transactions_data) >= total_tx:
        break

print(f" → Retrieved {len(transactions_data)} total transactions.")

identity_resp = plaid_post("/identity/get", auth_payload)
if identity_resp and "accounts" in identity_resp:
    identity_accounts = identity_resp["accounts"]
    identity_owners = []
    for acct in identity_accounts:
        for owner in acct.get("owners", []):
            identity_owners.append(owner)
    identity_data = {"owners": identity_owners}
    print(f" → Retrieved identity info for {len(identity_owners)} owner(s).")
else:
    identity_data = None
    print(" → No identity data available.")


holdings_resp = plaid_post("/investments/holdings/get", auth_payload)
if holdings_resp and "holdings" in holdings_resp:
    holdings_data = holdings_resp["holdings"]
    securities_data = holdings_resp["securities"]
    investment_accounts = holdings_resp["accounts"]
    print(f" → Retrieved {len(holdings_data)} holdings across {len(investment_accounts)} investment account(s).")
else:
    holdings_data = []
    securities_data = []
    investment_accounts = []
    print(" → No investment holdings data found (may not be enabled).")

inv_transactions_data = []
inv_offset = 0
while True:
    inv_tx_payload = {
        **auth_payload,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "options": {"count": page_size, "offset": inv_offset}
    }
    inv_tx_resp = plaid_post("/investments/transactions/get", inv_tx_payload)
    if not inv_tx_resp or "investment_transactions" not in inv_tx_resp:
        break
    batch = inv_tx_resp.get("investment_transactions", [])
    total_inv_tx = inv_tx_resp.get("total_investment_transactions", len(batch))
    inv_transactions_data.extend(batch)
    inv_offset += len(batch)

    if len(inv_transactions_data) >= total_inv_tx:
        break

print(f" → Retrieved {len(inv_transactions_data)} investment transactions total.")


liabilities_resp = plaid_post("/liabilities/get", auth_payload)
if liabilities_resp and "liabilities" in liabilities_resp:
    liabilities_data = liabilities_resp["liabilities"]
    print(" → Retrieved liabilities data.")
else:
    liabilities_data = None
    print(" → No liabilities data available.")


income_resp = plaid_post("/income/get", auth_payload)
if income_resp and "income" in income_resp:
    income_data = income_resp["income"]
    print(" → Retrieved income data.")
else:
    income_data = None
    print(" → No income data available.")


asset_report_json = None

all_data = {
    "accounts": accounts_data,
    "transactions": transactions_data,
    "identity": identity_data,
    "investments": {
        "accounts": investment_accounts,
        "holdings": holdings_data,
        "securities": securities_data,
        "investment_transactions": inv_transactions_data
    },
    "liabilities": liabilities_data,
    "income": income_data,
    "asset_report": asset_report_json
}


with open("plaid_data.json", "w") as f:
    json.dump(all_data, f, indent=2)
print("Saved full dataset to 'plaid_data.json' in this Colab environment.")

if accounts_data:
    df_accounts = pd.json_normalize(accounts_data)
    df_accounts.to_csv("accounts.csv", index=False)
    print("Saved 'accounts.csv' for account data.")

if transactions_data:
    df_transactions = pd.json_normalize(transactions_data)
    df_transactions.to_csv("transactions.csv", index=False)
    print("Saved 'transactions.csv' for transaction data.")


print("\n--- ACCOUNTS ---")
pprint(all_data["accounts"], sort_dicts=False)

print("\n--- TRANSACTIONS ---")
pprint(all_data["transactions"], sort_dicts=False)

print("\n--- IDENTITY ---")
pprint(all_data["identity"], sort_dicts=False)

print("\n--- INVESTMENTS (ACCOUNTS) ---")
pprint(all_data["investments"]["accounts"], sort_dicts=False)

print("\n--- INVESTMENTS (HOLDINGS) ---")
pprint(all_data["investments"]["holdings"], sort_dicts=False)

print("\n--- INVESTMENTS (SECURITIES) ---")
pprint(all_data["investments"]["securities"], sort_dicts=False)

print("\n--- INVESTMENTS (TRANSACTIONS) ---")
pprint(all_data["investments"]["investment_transactions"], sort_dicts=False)

print("\n--- LIABILITIES ---")
pprint(all_data["liabilities"], sort_dicts=False)

print("\n--- INCOME ---")
pprint(all_data["income"], sort_dicts=False)

print("\n--- ASSET REPORT ---")
pprint(all_data["asset_report"], sort_dicts=False)

Streaming output truncated to the last 5000 lines.
  'pending': False,
  'pending_transaction_id': None,
  'personal_finance_category': {'confidence_level': 'LOW',
                                'detailed': 'ENTERTAINMENT_SPORTING_EVENTS_AMUSEMENT_PARKS_AND_MUSEUMS',
                                'primary': 'ENTERTAINMENT'},
  'personal_finance_category_icon_url': 'https://plaid-category-icons.plaid.com/PFC_ENTERTAINMENT.png',
  'transaction_code': None,
  'transaction_id': 'zqZ81RmPXwHydx9yjzNmhyQzPy6ZdJulBwX3E',
  'transaction_type': 'place',
  'unofficial_currency_code': None,
  'website': None},
 {'account_id': 'k5KaoLy9WQSMAq3MDGJlUkDVz4EJKEiLdyWdp',
  'account_owner': None,
  'amount': 2078.5,
  'authorized_date': None,
  'authorized_datetime': None,
  'category': ['Payment'],
  'category_id': '16000000',
  'check_number': None,
  'counterparties': [],
  'date': '2023-08-07',
  'datetime': None,
  'iso_currency_code': 'USD',
  'location': {'address': None,
               'city